<a href="https://colab.research.google.com/github/meheer-shukla/sentiment-analysis/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import random as r 
class Sentiment:
  NEGATIVE = "NEGATIVE"
  NEUTRAL = "NEUTRAL"
  POSITIVE = "POSITIVE"
class Review:
  def __init__(self ,text , score):
    self.text = text
    self.score = score
    self.sentiment = self.get_sentiment()
  def get_sentiment(self):
    if self.score <= 2 :
      return Sentiment.NEGATIVE
    elif self.score == 3:
      return Sentiment.NEUTRAL
    else:
      return Sentiment.POSITIVE
class ReviewContainer :
  def __init__(self , reviews) :
    self.reviews = reviews
  
  def get_text(self):
    return [x.text for x in self.reviews] 
  
  def get_sentiment(self):
    return[x.sentiment for x in self.reviews]
  
  def evenly_distribute(self):
    negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
    positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
    positive_shrunk = positive[:len(negative)]
    self.reviews = negative + positive_shrunk 
    r.shuffle(self.reviews)

In [25]:

import json 
filename = '/content/drive/MyDrive/Learning Data/Books_small_10000.json'

reviews = []

with open(filename) as f:
  for line in f:
    review = json.loads(line)
    reviews.append(Review(review['reviewText'],review['overall']))
reviews[5].sentiment


'POSITIVE'

In [37]:
from sklearn.model_selection import train_test_split

training , test = train_test_split(reviews , test_size=0.33 , random_state=42)

train_container = ReviewContainer(training)

test_container = ReviewContainer(test)


In [66]:
train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()


test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()



train_y.count(Sentiment.POSITIVE)
train_y.count(Sentiment.NEGATIVE)

436

BAG OF WORDS

In [67]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vectors[0].toarray())


The book was delivered on time and in an excellent condition.  For anyone who wants to read supporting argument for Islamic thinking, this book is very helpful to make one see what others see from the way society should be.
[[0 0 0 ... 0 0 0]]


CLASSIFICATION

In [68]:
from sklearn import svm

clf_svm = svm.SVC(kernel = 'linear')  

clf_svm.fit(train_x_vectors , train_y)

test_x[0]

clf_svm.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

In [69]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()

clf_dec.fit(train_x_vectors , train_y)

clf_dec.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

In [70]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors , train_y)

clf_log.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

In [74]:
print(clf_log.score(test_x_vectors, test_y) * 100)

81.49038461538461


In [75]:
from sklearn.metrics import f1_score
f1_score(test_y , clf_log.predict(test_x_vectors), average = None, labels = [Sentiment.POSITIVE , Sentiment.NEGATIVE])

array([0.82051282, 0.808933  ])

In [78]:
test_set = ['i loved it ' , 'i hated it' , 'this product is bad' , 'this product is amazing']
new_test = vectorizer.transform(test_set)

clf_log.predict(new_test)

array(['POSITIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE'], dtype='<U8')